In [ ]:
# Установка необходимых библиотек
!pip install -qU lightgbm --upgrade
!pip install -qU optuna scikit-learn implicit sentence-transformers

import importlib, sys, os, gc, time, math, random, datetime, warnings
import numpy as np, pandas as pd, torch, lightgbm as lgb, optuna, sklearn
from tqdm import tqdm

print(f" Библиотеки загружены. scikit-learn версия: {sklearn.__version__}")
print(f"GPU: {'есть' if torch.cuda.is_available() else 'нету'}")
print(f"Torch version: {torch.__version__}")

def check_lightgbm_gpu():
    try:
        import lightgbm as lgb
        data = np.random.rand(50, 2)
        label = np.random.randint(2, size=50)
        train_data = lgb.Dataset(data, label=label)
        params = {'objective': 'binary', 'device': 'gpu', 'verbose': -1}
        booster = lgb.train(params, train_data, num_boost_round=1)
        return True
    except Exception as e:
        print(f"GPU для LightGBM недоступен: {str(e)}")
        return False

lgb_gpu_available = check_lightgbm_gpu()
print(f"🚀 LightGBM GPU support: {'✅' if lgb_gpu_available else '❌'}")

if not hasattr(sklearn.base, '_fit_context'):
    def _fit_context(*args, **kwargs):
        return args[0]
    sklearn.base.BaseEstimator._fit_context = _fit_context

import os
import gc
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import lightgbm as lgb
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def set_all_seeds(seed=42):
    """Фиксация всех случайных генераторов"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f" Все сиды зафиксированы: {seed}")

set_all_seeds(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" GPU Type: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f" GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB" if torch.cuda.is_available() else "N/A")
print(f" Device: {device}")

data_dir = '/kaggle/input/qwerty/'                         #УСТАНОВИТЬ ЗДЕСЬ ВАШ ПУТЬ
for path in ['/kaggle/input/qwerty/','/kaggle/input/']:          #УСТАНОВИТЬ ЗДЕСЬ ВАШ ПУТЬ
    if os.path.exists(os.path.join(path, 'train.csv')):
        data_dir = path
        break

print(f" Используемый путь к данным: {data_dir}")

train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))
books_df = pd.read_csv(os.path.join(data_dir, 'books.csv'))
users_df = pd.read_csv(os.path.join(data_dir, 'users.csv'))
book_desc_df = pd.read_csv(os.path.join(data_dir, 'book_descriptions.csv'))

print(f" Размеры данных:")
print(f"   Train: {train_df.shape}")
print(f"   Test: {test_df.shape}")
print(f"   Books: {books_df.shape}")
print(f"   Users: {users_df.shape}")
print(f"   Descriptions: {book_desc_df.shape}")

original_test_user_ids = test_df['user_id'].copy().values
original_test_book_ids = test_df['book_id'].copy().values

train_ratings = train_df[train_df['has_read'] == 1].copy()
train_ratings['timestamp'] = pd.to_datetime(train_ratings['timestamp'])

print(" Объединение данных...")

train_merged = train_ratings.merge(books_df, on='book_id', how='left')
train_merged = train_merged.merge(users_df, on='user_id', how='left')
test_merged = test_df.merge(books_df, on='book_id', how='left')
test_merged = test_merged.merge(users_df, on='user_id', how='left')

train_merged = train_merged.merge(book_desc_df[['book_id', 'description']], on='book_id', how='left')
test_merged = test_merged.merge(book_desc_df[['book_id', 'description']], on='book_id', how='left')

print(f" Объединенные данные:")
print(f"   Train merged: {train_merged.shape}")
print(f"   Test merged: {test_merged.shape}")

print(" Генерация текстовых признаков(TF-IDF)...")

train_merged['description'] = train_merged['description'].fillna('')
test_merged['description'] = test_merged['description'].fillna('')

tfidf = TfidfVectorizer(
    max_features=800,  
    ngram_range=(1, 2),  
    min_df=3,
    max_df=0.95,
    sublinear_tf=True,
    norm='l2'
)
tfidf_train = tfidf.fit_transform(train_merged['description'])
tfidf_test = tfidf.transform(test_merged['description'])

tfidf_cols = [f'tfidf_{i}' for i in range(tfidf_train.shape[1])]
train_tfidf_df = pd.DataFrame(tfidf_train.toarray(), columns=tfidf_cols)
test_tfidf_df = pd.DataFrame(tfidf_test.toarray(), columns=tfidf_cols)

print(f" TF-IDF создано: {len(tfidf_cols)} признаков")

train_merged = pd.concat([
    train_merged.reset_index(drop=True), 
    train_tfidf_df.reset_index(drop=True)
], axis=1)

test_merged = pd.concat([
    test_merged.reset_index(drop=True),
    test_tfidf_df.reset_index(drop=True)
], axis=1)

print(f" Всего текстовых признаков: {len(tfidf_cols)}")
print(f" Размер train_merged после добавления текстовых признаков: {train_merged.shape}")

def create_advanced_features(df, train_df=None):
    if 'timestamp' in df.columns:
        df['hour'] = df['timestamp'].dt.hour
        df['day_of_week'] = df['timestamp'].dt.dayofweek
        df['month'] = df['timestamp'].dt.month
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        df['quarter'] = df['timestamp'].dt.quarter
    
    if train_df is not None:
        book_popularity = train_df.groupby('book_id')['timestamp'].agg(
            ['count', 'max', 'min']
        ).rename(columns={'count': 'book_total_ratings', 'max': 'book_last_rating', 'min': 'book_first_rating'})
        
        df = df.merge(book_popularity, on='book_id', how='left')
        
        user_activity = train_df.groupby('user_id')['timestamp'].agg(
            ['count', 'max', 'min']
        ).rename(columns={'count': 'user_total_ratings', 'max': 'user_last_rating', 'min': 'user_first_rating'})
        
        df = df.merge(user_activity, on='user_id', how='left')
        
        if 'timestamp' in df.columns and 'user_last_rating' in df.columns:
            df['days_since_user_last_rating'] = (df['timestamp'] - pd.to_datetime(df['user_last_rating'])).dt.days
            df['days_since_user_last_rating'] = df['days_since_user_last_rating'].fillna(30)
            df['days_since_user_first_rating'] = (df['timestamp'] - pd.to_datetime(df['user_first_rating'])).dt.days
        
        if 'timestamp' in df.columns and 'book_last_rating' in df.columns:
            df['days_since_book_last_rating'] = (df['timestamp'] - pd.to_datetime(df['book_last_rating'])).dt.days
            df['days_since_book_last_rating'] = df['days_since_book_last_rating'].fillna(30)
            df['days_since_book_first_rating'] = (df['timestamp'] - pd.to_datetime(df['book_first_rating'])).dt.days
    
    if 'user_avg_rating' in df.columns and 'book_avg_rating' in df.columns:
        df['user_book_rating_diff'] = df['user_avg_rating'] - df['book_avg_rating']
        df['user_book_rating_ratio'] = df['user_avg_rating'] / (df['book_avg_rating'] + 1e-8)
        df['user_book_rating_product'] = df['user_avg_rating'] * df['book_avg_rating']
    
    if 'user_rating_count' in df.columns and 'book_rating_count' in df.columns:
        df['user_book_popularity_ratio'] = df['user_rating_count'] / (df['book_rating_count'] + 1e-8)
        df['user_book_popularity_diff'] = df['user_rating_count'] - df['book_rating_count']
    
    if 'description' in df.columns:
        df['description_length'] = df['description'].fillna('').apply(len)
        df['has_description'] = (df['description'].fillna('') != '').astype(int)
        df['description_word_count'] = df['description'].fillna('').str.split().str.len()
    
    return df

def create_aggregated_features(train_df, target_df):
    user_stats = train_df.groupby('user_id')['rating'].agg(
        ['mean', 'count', 'std', 'median', 'max', 'min']
    ).rename(columns={
        'mean': 'user_avg_rating',
        'count': 'user_rating_count',
        'std': 'user_rating_std',
        'median': 'user_median_rating',
        'max': 'user_max_rating',
        'min': 'user_min_rating'
    }).fillna(0).reset_index()
    
    book_stats = train_df.groupby('book_id')['rating'].agg(
        ['mean', 'count', 'std', 'median', 'max', 'min']
    ).rename(columns={
        'mean': 'book_avg_rating',
        'count': 'book_rating_count',
        'std': 'book_rating_std',
        'median': 'book_median_rating',
        'max': 'book_max_rating',
        'min': 'book_min_rating'
    }).fillna(0).reset_index()
    
    author_stats = train_df.groupby('author_id')['rating'].agg(
        ['mean', 'count', 'std', 'median']
    ).rename(columns={
        'mean': 'author_avg_rating',
        'count': 'author_rating_count',
        'std': 'author_rating_std',
        'median': 'author_median_rating'
    }).fillna(0).reset_index()
    
    target_df = target_df.merge(user_stats, on='user_id', how='left')
    target_df = target_df.merge(book_stats, on='book_id', how='left')
    target_df = target_df.merge(author_stats, on='author_id', how='left')
    
    for col in ['user_avg_rating', 'book_avg_rating', 'author_avg_rating',
                'user_median_rating', 'book_median_rating', 'author_median_rating']:
        target_df[col] = target_df[col].fillna(target_df[col].mean())
    
    for col in ['user_rating_count', 'book_rating_count', 'author_rating_count',
                'user_rating_std', 'book_rating_std', 'author_rating_std']:
        target_df[col] = target_df[col].fillna(0)
    
    return target_df

train_merged = train_merged.sort_values('timestamp')
split_date = train_merged.iloc[int(len(train_merged) * 0.8)]['timestamp']
train_data = train_merged[train_merged['timestamp'] <= split_date].copy()
valid_data = train_merged[train_merged['timestamp'] > split_date].copy()

print(f" Разделение данных по времени:")
print(f"   Split date: {split_date}")
print(f"   Train  {train_data.shape}")
print(f"   Valid  {valid_data.shape}")

print(" Создание признаков...")
train_data = create_advanced_features(train_data)
valid_data = create_advanced_features(valid_data, train_data)
test_data = create_advanced_features(test_merged, train_merged)

train_data = create_aggregated_features(train_data, train_data.copy())
valid_data = create_aggregated_features(train_data, valid_data.copy())
test_data = create_aggregated_features(train_merged, test_data.copy())

print(" Признаки созданы для всех датасетов")

base_features = [
    'user_avg_rating', 'user_rating_count', 'user_rating_std', 'user_median_rating',
    'user_max_rating', 'user_min_rating',
    'book_avg_rating', 'book_rating_count', 'book_rating_std', 'book_median_rating',
    'book_max_rating', 'book_min_rating',
    'author_avg_rating', 'author_rating_count', 'author_rating_std', 'author_median_rating',
    'publication_year', 'avg_rating', 'page_count',
    'hour', 'day_of_week', 'month', 'is_weekend', 'quarter',
    'book_total_ratings', 'user_total_ratings',
    'days_since_user_last_rating', 'days_since_book_last_rating',
    'days_since_user_first_rating', 'days_since_book_first_rating',
    'user_book_rating_diff', 'user_book_rating_ratio', 'user_book_rating_product',
    'user_book_popularity_ratio', 'user_book_popularity_diff',
    'description_length', 'has_description', 'description_word_count'
]

tfidf_features = [col for col in train_data.columns if 'tfidf_' in col]

all_features = base_features + tfidf_features

final_features = [col for col in all_features if col in train_data.columns and col in test_data.columns]

print(f" Используемые признаки ({len(final_features)}):")
print(f"    Базовые: {len(base_features)}")
print(f"    TF-IDF: {len(tfidf_features)}")

X_train = train_data[final_features].copy()
y_train = train_data['rating']
X_valid = valid_data[final_features].copy()
y_valid = valid_data['rating']
X_test = test_data[final_features].copy()

for col in final_features:
    if X_train[col].isna().any() or X_valid[col].isna().any() or X_test[col].isna().any():
        mean_val = X_train[col].mean()
        X_train[col] = X_train[col].fillna(mean_val)
        X_valid[col] = X_valid[col].fillna(mean_val)
        X_test[col] = X_test[col].fillna(mean_val)

for col in final_features:
    for df in [X_train, X_valid, X_test]:
        if np.isinf(df[col]).any():
            max_val = df[col][np.isfinite(df[col])].max()
            min_val = df[col][np.isfinite(df[col])].min()
            df[col] = df[col].replace([np.inf, -np.inf], [max_val, min_val])

scaler = RobustScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=final_features)
X_valid = pd.DataFrame(scaler.transform(X_valid), columns=final_features)
X_test = pd.DataFrame(scaler.transform(X_test), columns=final_features)

print(" Предобработка признаков завершена")

def train_als_features(train_df, user_col='user_id', item_col='book_id', rating_col='rating', factors=64):
    try:
        from implicit.als import AlternatingLeastSquares
        from scipy.sparse import csr_matrix
        import scipy.sparse as sparse
        
        print(" Обучение ALS...")
        
        # Создаем копию чтобы не менять оригинальные данные
        train_df = train_df.copy()
        
        # Проверка версии implicit
        try:
            import implicit
            print(f" implicit версия: {implicit.__version__}")
        except:
            print(" Не удалось определить версию implicit")
        
        alpha = 40.0 
        epsilon = 1e-8
        
        min_rating = train_df[rating_col].min()
        max_rating = train_df[rating_col].max()
        if max_rating > min_rating:
            train_df['normalized_rating'] = (train_df[rating_col] - min_rating) / (max_rating - min_rating)
        else:
            train_df['normalized_rating'] = 0.5
        
        train_df['confidence'] = 1.0 + alpha * train_df['normalized_rating']
        
        all_users = pd.concat([train_df[user_col], valid_data[user_col]]).unique()
        all_items = pd.concat([train_df[item_col], valid_data[item_col]]).unique()
        
        all_users = sorted(all_users)
        all_items = sorted(all_items)
        
        user_to_idx = {user: idx for idx, user in enumerate(all_users)}
        item_to_idx = {item: idx for idx, item in enumerate(all_items)}
        
        train_df['user_idx'] = train_df[user_col].map(user_to_idx).fillna(-1).astype(int)
        train_df['item_idx'] = train_df[item_col].map(item_to_idx).fillna(-1).astype(int)
        
        valid_mask = (train_df['user_idx'] >= 0) & (train_df['item_idx'] >= 0)
        train_df = train_df[valid_mask].copy()
        
        print(f" Данные для ALS: {len(train_df)} взаимодействий, "
              f"{len(all_users)} пользователей, {len(all_items)} книг")
        
        sparse_matrix = csr_matrix(
            (train_df['confidence'].values, 
             (train_df['user_idx'].values, train_df['item_idx'].values)),
            shape=(len(all_users), len(all_items)),
            dtype=np.float32
        )
        
        use_gpu = False
        try:
            if torch.cuda.is_available():
                from implicit import gpu
                has_gpu = gpu.HAS_CUDA
                if has_gpu:
                    use_gpu = True
                    print(" GPU доступен для ALS")
                else:
                    print(" GPU недоступен для ALS, используем CPU")
            else:
                print(" CUDA недоступна, используем CPU для ALS")
        except Exception as e_gpu:
            print(f" Ошибка проверки GPU для ALS: {e_gpu}, используем CPU")
    
        model = AlternatingLeastSquares(
            factors=factors,
            regularization=0.01,  
            iterations=20,       
            use_gpu=use_gpu,
            num_threads=4,       
            random_state=42,      
            dtype=np.float32
        )
        
        start_time = time.time()
        
        try:
            model.fit(sparse_matrix.T, show_progress=False)  
        except TypeError:
       
            model.fit(sparse_matrix.T)
        
        training_time = time.time() - start_time
        print(f" ALS успешно обучен за {training_time:.2f} секунд")
        
        user_embeddings = model.user_factors.toarray() if hasattr(model.user_factors, 'toarray') else model.user_factors
        item_embeddings = model.item_factors.toarray() if hasattr(model.item_factors, 'toarray') else model.item_factors
        
        print(f" Размер эмбеддингов: пользователи={user_embeddings.shape}, книги={item_embeddings.shape}")
        
        if np.isnan(user_embeddings).any() or np.isnan(item_embeddings).any():
            print(" Обнаружены NaN в эмбеддингах, переключаемся на CPU версию")
            raise ValueError("NaN в эмбеддингах")
        
        return user_embeddings, item_embeddings, user_to_idx, item_to_idx
        
    except Exception as e:
        print(f" Ошибка ALS: {str(e)}")
        
        print(" Создание fallback признаков популярности...")
        
        book_popularity = train_df.groupby('book_id')['rating'].mean().to_dict()
        user_activity = train_df.groupby('user_id')['rating'].count().to_dict()
        
        all_users = train_df[user_col].unique()
        all_items = train_df[item_col].unique()
        
        user_embeddings = np.zeros((len(all_users), factors))
        item_embeddings = np.zeros((len(all_items), factors))
        
        for i, user in enumerate(all_users):
            if user in user_activity:
                user_embeddings[i, 0] = user_activity[user]
        
        for i, item in enumerate(all_items):
            if item in book_popularity:
                item_embeddings[i, 0] = book_popularity[item]
        
        user_to_idx = {user: i for i, user in enumerate(all_users)}
        item_to_idx = {item: i for i, item in enumerate(all_items)}
        
        print(" Созданы fallback признаки популярности")
        return user_embeddings, item_embeddings, user_to_idx, item_to_idx

user_embs, item_embs, user_to_idx, item_to_idx = train_als_features(
    train_data[['user_id', 'book_id', 'rating']].copy(),
    factors=64
)

if user_embs is not None and item_embs is not None:
    train_data['user_idx'] = train_data['user_id'].map(user_to_idx).fillna(-1).astype(int)
    train_data['item_idx'] = train_data['book_id'].map(item_to_idx).fillna(-1).astype(int)
    
    als_user_features = []
    als_item_features = []
    
    for i in range(64):
        train_data[f'als_user_{i}'] = train_data['user_idx'].apply(
            lambda x: user_embs[x, i] if x >= 0 and x < user_embs.shape[0] else 0
        )
        train_data[f'als_item_{i}'] = train_data['item_idx'].apply(
            lambda x: item_embs[x, i] if x >= 0 and x < item_embs.shape[0] else 0
        )
        als_user_features.append(f'als_user_{i}')
        als_item_features.append(f'als_item_{i}')
    
    valid_data['user_idx'] = valid_data['user_id'].map(user_to_idx).fillna(-1).astype(int)
    valid_data['item_idx'] = valid_data['book_id'].map(item_to_idx).fillna(-1).astype(int)
    
    for i in range(64):
        valid_data[f'als_user_{i}'] = valid_data['user_idx'].apply(
            lambda x: user_embs[x, i] if x >= 0 and x < user_embs.shape[0] else 0
        )
        valid_data[f'als_item_{i}'] = valid_data['item_idx'].apply(
            lambda x: item_embs[x, i] if x >= 0 and x < item_embs.shape[0] else 0
        )
    
    test_data['user_idx'] = test_data['user_id'].map(user_to_idx).fillna(-1).astype(int)
    test_data['item_idx'] = test_data['book_id'].map(item_to_idx).fillna(-1).astype(int)
    
    for i in range(64):
        test_data[f'als_user_{i}'] = test_data['user_idx'].apply(
            lambda x: user_embs[x, i] if x >= 0 and x < user_embs.shape[0] else 0
        )
        test_data[f'als_item_{i}'] = test_data['item_idx'].apply(
            lambda x: item_embs[x, i] if x >= 0 and x < item_embs.shape[0] else 0
        )
    
    final_features.extend(als_user_features + als_item_features)
    
    print(f" Добавлено ALS признаков: {len(als_user_features) + len(als_item_features)}")
    
    X_train = train_data[final_features].copy()
    X_valid = valid_data[final_features].copy()
    X_test = test_data[final_features].copy()
    
    scaler = RobustScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=final_features)
    X_valid = pd.DataFrame(scaler.transform(X_valid), columns=final_features)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=final_features)

train_data_lgb = lgb.Dataset(X_train, label=y_train)
valid_data_lgb = lgb.Dataset(X_valid, label=y_valid, reference=train_data_lgb)

print(f" LightGBM датасеты подготовлены:")
print(f"   Train samples: {X_train.shape[0]}")
print(f"   Features: {X_train.shape[1]}")

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 0.95),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.8, 0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 5, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.5),
        'max_bin': 255,
        'feature_pre_filter': False,
        'verbosity': -1,
        'random_state': 42,
        'n_jobs': -1,
        'device': 'gpu' if lgb_gpu_available else 'cpu',
    }
    
    model = lgb.train(
        params,
        train_data_lgb,
        num_boost_round=2000,
        valid_sets=[valid_data_lgb],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0)
        ]
    )
    
    y_pred = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    return rmse

print(" Оптимизация гиперпараметров...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1,
    'device': 'gpu' if lgb_gpu_available else 'cpu',
    'feature_pre_filter': False
})

print(f" Лучшие параметры: {best_params}")
print(f" Лучший RMSE: {study.best_value:.4f}")

print(" Обучение финальной модели...")
final_model = lgb.train(
    best_params,
    train_data_lgb,
    num_boost_round=5000,
    valid_sets=[valid_data_lgb],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100, verbose=True),
        lgb.log_evaluation(period=50)
    ]
)

y_pred_valid = final_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print(f" Финальный RMSE на валидации: {rmse:.4f}")

test_preds = final_model.predict(X_test)
test_preds = np.clip(test_preds, 0, 10)

min_size = min(len(test_preds), len(original_test_user_ids), len(original_test_book_ids))
submission_df = pd.DataFrame({
    'user_id': original_test_user_ids[:min_size],
    'book_id': original_test_book_ids[:min_size],
    'rating_predict': test_preds[:min_size]
})

submission_df['rating_predict'] = submission_df['rating_predict'].fillna(5.0)

output_dir = '/kaggle/working/'         #УСТАНОВИТЬ ВАШ ПУТЬ КУДА БУДЕТ СОЗДАН САБМИТ
os.makedirs(output_dir, exist_ok=True)
submission_path = os.path.join(output_dir, 'submission2.csv')
submission_df.to_csv(submission_path, index=False)

print(f" Submission сохранен: {submission_path}")
print(f" Финальная статистика: min={submission_df['rating_predict'].min():.4f}, max={submission_df['rating_predict'].max():.4f}, mean={submission_df['rating_predict'].mean():.4f}")

gc.collect()
torch.cuda.empty_cache()
print("\n Все этапы успешно завершены!")
